# Win rate 

Having established what are our respective champion pools and loaded them into a table I can begin the analysis from simply determining our champion winrate. It will give my friends and myself a reqlity check on which of our favorite champions are currently the best choice. Fan-favorite statistic is a specific's champion win rate meaning percentage of game swon with that champion in relation to all games played by that champion. With hundred of thousands of League's players playing every day there is a good chance that after only a few days we have enough data to adapt an assumption that winrate does not occur due to some random events but rather a systematic effect of changes made by game's development team that adjust champions and items in game. 

## Data used in the notebook

In this analysis we use data from real matches in ranked soloq leagues from Platinum IV to Diamond I. Data is pulled from Riot Developer API endpoints using a development key. It was achieved by pulling data from various endpoints, tranforming their responses and based on that reposonses I have been able to create a table of random 10,000 games played since last patch (changes made to game that can potentialy impact champion winrate). All technical are desribed in comments of files contained in the directory `api_calls`. 

That resulted in a table of 100,000 records from those 10,000 games. Each record represents a champion played suplemented by information on whether game has been won by the champion or lost. Based on that information we will be able to calculate some summary statistics and 

## Package import
Import of all custom function packages that will be used in this notebook

In [1]:
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import sqlite3


## Load and transform data
Data extracted from official game developer can now be load into pandas dataframes (tables if you wish) to be used in my calculations. But first we need a connection to previously created database in order to access a data stored in tables. 

### Connect to a database
I order to allow interaction between Python and our database we need to establish a connection and create a cursor object. 

In [3]:
main_db = sqlite3.connect(r'/Users/amberlights/Repositories/simple-lol-stats/sqlite_database/project_database.db')
cursor = main_db.cursor()

### Summary Statistics

Let's calculate them directly in sql as a gigachad data analyst. 

In [4]:
highest_winrate_10 = pd.read_sql('''
SELECT champion, ROUND(AVG(have_won), 2) AS win_rate, count(champion) as games_played
FROM winrate_data
GROUP BY champion
ORDER BY AVG(have_won) DESC
LIMIT 10;
''', main_db)

highest_play_count_10 = pd.read_sql('''
SELECT champion, ROUND(AVG(have_won), 2) AS win_rate, count(champion) as games_played
FROM winrate_data
GROUP BY champion
ORDER BY count(champion) DESC
LIMIT 10;
''', main_db)

and display using Jupyter's wonderful possibility of not using 'print()'

### 10 Most frequently played champions

In [6]:
highest_play_count_10

,champion,win_rate,games_played
0,Jinx,0.50,2034
1,Caitlyn,0.51,2025
2,Lux,0.61,2012
3,MonkeyKing,0.61,1940
4,Lulu,0.53,1743
5,MissFortune,0.52,1557
6,Jhin,0.48,1547
7,Viego,0.52,1462
8,TahmKench,0.35,1420
9,Nami,0.52,1397


### 10 Champions with the highest winrate

In [7]:
highest_winrate_10

,champion,win_rate,games_played
0,Sona,0.79,679
1,Chogath,0.69,1144
2,Vayne,0.68,1269
3,MonkeyKing,0.61,1940
4,Lux,0.61,2012
5,Anivia,0.60,233
6,Nidalee,0.58,150
7,Rumble,0.58,119
8,Singed,0.57,216
9,RekSai,0.57,139


## Visualisation
After a glimpse of what can we expect in our data let's prepare the actuals that we will use in our exercise. 

### Data need for plots
Data pulled from API representing winrate for every champion played since last patch in ranked games on EUNE server

In [41]:
server_winrate = pd.read_sql('''
SELECT c.name as champion, ROUND(AVG(w.have_won), 4) AS winrate, count(w.champion) as games_played
FROM winrate_data w
LEFT JOIN champions c
ON w.champion = c.id
WHERE champion is not null
GROUP BY c.name;
''', main_db
)

Data of our team champion pools and roles

In [35]:
npb_champion_pool = pd.read_sql('''
SELECT role, champion
FROM champion_pool
''', main_db
)

### Data transformation

Having both sets read as pandas data frames, we can connect them together using left join - meaing that for all data in our champion pool table we will 'add' a data from a `server_winrate` table. 

In [42]:
data_transformed = pd.merge(npb_champion_pool, server_winrate, on='champion', how='left')

now lets change winrate from decimal to percentage

In [43]:
data_transformed['winrate'] = data_transformed['winrate']*100
data_transformed.head()

,role,champion,winrate,games_played
0,jungler,Lillia,21.71,737
1,jungler,Volibear,51.41,817
2,jungler,Sejuani,49.81,269
3,jungler,Poppy,53.10,403
4,jungler,Ekko,49.91,587


It is hard to believe that Lillia has so low winrate! I love that champ. Is it possible that she got so hard nerfs last patch so she is actually tht weak right now?
IF so - its good thing I decided to check instead of blindly playing her in clash but that number seems so low so almost impossible. Lets make sure it is not some problem with our calculations. 

### Lillia test case

In [27]:
lillia = pd.read_sql('''
SELECT count(champion) AS games, have_won
FROM winrate_data
WHERE champion = 'Lillia'
GROUP BY have_won;
''', main_db
)
lillia

,games,have_won
0,577,0
1,160,1


### Test conclusion
Unfortunately the initial info was not missleading. Lillia apparently took a huge hit due to recent changes. Sadly as it may be I have to decide not to play her this weekend. 

PS. That could have been obviously easily achieved directly in SQL using nested query (as displayed below) but I elected to stick to pandas for easier step by step explanation of my logic. 

In [ ]:
SELECT w.*, p.role
FROM (SELECT c.name as champion, ROUND(AVG(w.have_won)*100, 2) AS winrate, count(w.champion) as games_played
    FROM winrate_data w
    LEFT JOIN champions c
    ON w.champion = c.id
    WHERE champion_name is not null
    GROUP BY c.name) w
LEFT JOIN champion_pool p
ON w.champion = p.champion
WHERE p.role is not null;

In [54]:
data_colors = data_transformed.sort_values(by='champion')

colors = sns.color_palette('rocket', data_colors['champion'].nunique()).as_hex()
color_map = dict(zip(data_colors['champion'].unique(), colors))

In [56]:
from plotly.subplots import make_subplots

def create_role_plots(df, role_order, plot_height=300, plot_width=800):
    fig = make_subplots(
        rows=len(role_order), cols=1,
        subplot_titles=[f"{role.capitalize()}" for role in role_order],
        vertical_spacing=0.06
    )

    for i, role in enumerate(role_order):
        df_role = df[df['role'] == role]
        fig.add_trace(
            go.Bar(
                x=df_role['champion'],
                y=df_role['winrate'],
                marker_color=[color_map[champ] for champ in df_role['champion']],
                name=role
            ),
            row=i+1, col=1
        )
        # Add dashed line at 50% win rate
        fig.add_shape(
            type='line',
            x0=-0.5, x1=len(df_role['champion'].unique())-0.5,
            y0=50.0, y1=50.0,
            line=dict(
                color='Black',
                width=2,
                dash='dashdot',
            ),
            row=i+1, col=1
        )

    fig.update_layout(
        title_text='Champion Win Rates by Role',
        height=plot_height * len(role_order),  # Adjust the height based on the number of roles
        width=plot_width,
        plot_bgcolor = '#F5E9DA', 
        paper_bgcolor ='#F5E9DA',
        showlegend=False, 
        font=dict(
            family='Roboto',
            size = 16
        )
    )

    fig.update_yaxes(title_text='Win Rate (%)', range=[0, 100.00], showgrid = False)
    fig.update_xaxes(title_text='Champion')

    return fig

In [57]:
role_order = ['top', 'jungler', 'mid', 'adc' ,'support']
fig = create_role_plots(data_transformed, role_order, plot_height=500, plot_width=1200)
fig.show()